In [3]:
import torch
import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Label
import numpy as np
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
print(f"Using Torch version {torch.__version__}.  CUDA is {'available' if torch.cuda.is_available() else 'not available'}.")
print(f"Using bokeh version {bokeh.__version__}.")
import pandas as pd
output_notebook()

/home/jet08013/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using Torch version 1.13.1.post200.  CUDA is available.
Using bokeh version 3.1.0.


Loading BokehJS ...

In [4]:
class LogR(torch.nn.Module):
    """A simple 15x1 logistic regression model."""
    def __init__(self):
        super().__init__()
        self.logistic=torch.nn.Linear(15,1,dtype=torch.float64)

    def forward(self,x):
        return torch.sigmoid(self.logistic(x))

In [5]:
df = pd.read_csv('ifood_df.csv',delimiter=',')
features = df.columns[[0,4,5,6,7,8,9,10,11,12,13,14,24,36,37]]
df[features] = (df[features]-df[features].mean())/df[features].std()
x_train, x_test, y_train, y_test = train_test_split(df[features].values, df['Response'].values)


In [6]:
model = LogR()
print(model)
for name,param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

LogR(
  (logistic): Linear(in_features=15, out_features=1, bias=True)
)
Layer: logistic.weight | Size: torch.Size([1, 15]) | Values : tensor([[-0.1446,  0.0861, -0.1988,  0.2049, -0.1940, -0.0374, -0.0060, -0.0183,
         -0.0475, -0.0228,  0.2516,  0.0330,  0.2302, -0.0207,  0.1922]],
       dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: logistic.bias | Size: torch.Size([1]) | Values : tensor([0.0055], dtype=torch.float64, grad_fn=<SliceBackward0>) 



In [7]:
Xt = torch.tensor(x_train,dtype=torch.float64)
Yt = torch.tensor(y_train,dtype=torch.float64)
criterion = torch.nn.functional.binary_cross_entropy
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)


In [8]:
def train(model, criterion, optimizer, Xt, Yt):
    """One step through the training loop"""
    # reset the gradient calculations
    optimizer.zero_grad()

    # forward pass
    predicted = model(Xt)
    
    # compute the loss
    loss = criterion(torch.squeeze(predicted),Yt)
    
 

    # compute the gradients by backward propogation
    loss.backward()        
        
    # adjust the weights
    optimizer.step()
    
    return loss.item()

In [9]:
def plot_loss(model, data, target, learning_rate=.0001, threshold=1e-6, max_iter=100000):
    """Run the model and collect the losses; return a figure"""
    
    criterion = torch.nn.functional.binary_cross_entropy
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    losses = []
    prior_loss=1000000
    for i in tqdm.tqdm(range(max_iter)):
        loss = train(model,criterion,optimizer,Xt,Yt)
        losses.append(loss)
        if abs(loss-prior_loss) < threshold:
            break
        prior_loss = loss

 
    f=figure(title=f"Loss over time: lr={learning_rate} threshold={threshold}",x_axis_label="Epoch",y_axis_label="Loss")
    f.line(x=list(range(len(losses))),y=losses)
    with torch.no_grad():
        prediction = model(Xt).round()
        print(f"Accuracy is {((prediction - Yt.reshape(-1,1))==0).sum()/Xt.shape[0]}")
    
    return f

In [10]:
model = LogR()
show(plot_loss(model, Xt, Yt))

 60%|██████    | 60444/100000 [00:15<00:10, 3907.83it/s]


Accuracy is 0.8554143905639648
